In [ ]:
#! pip install kaggle
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle competitions download -c iais22-birds
#! unzip iais22-birds.zip -d 'sample_data'


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.utils.data as data
from torchvision import datasets

In [19]:
mean = [0.4704, 0.4669, 0.3898]
std = [0.2035, 0.2001, 0.2047]
number_of_images = 58388

train_dataset_path = '../input/iais22-birds/birds/birds'
train_transforms = transforms.Compose ([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(), #Para que los datos no estén sesgados, le aplicamos modificaciones
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [20]:
orig_set = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = train_transforms)

In [48]:
#drop_last=True, pin_memory=True
n = len(orig_set)  # total number of examples
n_test = int(0.1 * n)  # take ~10% for test
train_set_size = int(n * 0.9)
valid_set_size = n - train_set_size
train_set, valid_set = data.random_split(orig_set, [train_set_size, valid_set_size])

train_loader = torch.utils.data.DataLoader(orig_set, batch_size = 64, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(valid_set, batch_size = 64, shuffle=False, pin_memory=True)

In [49]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

In [50]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for xi, yi in loader:
            xi = xi.to(device=device, dtype = torch.float32)
            yi = yi.to(device=device, dtype = torch.long)
            scores = model(xi) 
            _, pred = scores.max(dim=1) 
            num_correct += (pred == yi).sum() 
            num_total += pred.size(0)
        return float(num_correct)/num_total   

In [51]:
from datetime import datetime
def train(model, optimiser, epochs):
    model = model.to(device=device)
    for epoch in range(epochs):
        for i, (xi, yi) in enumerate(train_loader):
            model.train()
            xi = xi.to(device=device, dtype=torch.float32)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)
            cost = F.cross_entropy(input= scores, target=yi)
            optimiser.zero_grad()           
            cost.backward()
            optimiser.step()              
        acc = accuracy(model, train_loader) 
        val_acc = accuracy(model, test_loader) 
        date = datetime.now()
        print(f'Epoch: {epoch}, costo: {cost.item()}, accuracy: {acc}, val_accuracy: {val_acc}, hora: {date}')

In [52]:
class CNN_class1(nn.Module):
    def __init__(self, in_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channel, out_channels = 32,
                              kernel_size=3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64,
                              kernel_size=3, padding=1)
        self.max_pool1 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128,
                              kernel_size=3, padding = 1)
        self.max_pool2 = nn.MaxPool2d(2,2)



        self.fc = nn.Linear(in_features=401408, out_features=400) #El 32 es dependiendo del tamaño de la image
        self.flatten = nn.Flatten()
    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x))))
        x = self.max_pool1(x)
        x = F.relu(self.conv3(x))
        x = self.max_pool2(x)
        x = self.flatten(x)
        return self.fc(x)

In [54]:
epochs = 35
lr = 0.0001
modelCNN2 = CNN_class1(3)
optimiser = torch.optim.Adam(modelCNN2.parameters(), lr)

In [72]:
train(modelCNN2, optimiser, epochs)

In [ ]:
#GUARDAR MODELO
model_path = 'transformer2.pth'
torch.save(modelCNN2.state_dict(),model_path)

In [ ]:
class CNN_class1(nn.Module):
    def __init__(self, in_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channel, out_channels = 32,
                              kernel_size=3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64,
                              kernel_size=3, padding=1)
        self.max_pool1 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 92,
                              kernel_size=3, padding = 1)
        self.max_pool2 = nn.MaxPool2d(2,2)
        

        
        self.fc = nn.Linear(in_features=211968, out_features=400) #El 32 es dependiendo del tamaño de la image
        self.flatten = nn.Flatten()
    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x))))
        x = self.max_pool1(x)
        x = F.relu(self.conv3(x))
        x = self.max_pool2(x)
        x = self.flatten(x)
        return self.fc(x)

In [92]:
epochs = 30
lr = 0.00005
modelCNN23 = CNN_class1(3)
optimiser = torch.optim.Adam(modelCNN23.parameters(), lr)

In [ ]:
train(modelCNN23, optimiser, epochs)

In [ ]:
modelCNN23

In [ ]:
#--------------------------------------------------------------------------------------------------

In [41]:
modelCNN2

In [73]:
import glob
from pandas.core.common import flatten
from PIL import Image

import matplotlib.pyplot as plt

test_path = '../input/iais22-birds/submission_test/submission_test'
test_images_paths = []
for data_path in glob.glob(test_path + '/*'):
    test_images_paths.append(glob.glob(data_path))

test_images_paths = list(flatten(test_images_paths))
test_images_paths[1]

In [74]:
Categories = []
Id = []

for i in range (0, len(test_images_paths)):
    img_name = test_images_paths[i]
    x = img_name.replace('../input/iais22-birds/submission_test/submission_test/', '')
    x = x.split('.')
    Id.append(x[0])
    class_predict = predict_img_class(Image.open(img_name),modelCNN2,train_transforms)
    Categories.append(class_predict)
    
print(len(Categories))

In [ ]:
print(len(Id))

In [28]:
#orig_set.classes

In [75]:
def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)


train_transforms = transforms.Compose ([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(), #Para que no los datos no estén sesgados, le aplicamos modificaciones
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])


def predict_img_class(img,model,train_transforms):
    """ Predict the class of image and Return Predicted Class"""
    img = train_transforms(img)
    img = to_device(img.unsqueeze(0), device)
    prediction =  model(img)
    _, preds = torch.max(prediction, dim = 1)
    return orig_set.classes[preds[0].item()]

In [76]:
img = Image.open(test_images_paths[10])
print(test_images_paths[10])
plt.imshow(img)

In [88]:
print(f"Predicted Class : {predict_img_class(img,modelCNN2,train_transforms)}")


In [32]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [91]:
import pandas as pd

img_name = []
series_name = []


dict = {"Id": Id, "Category": Categories}

df = pd.DataFrame(dict)

df.to_csv('shows4.csv', index=False)

In [34]:
txt = '../input/iais22-birds/submission_test/submission_test/9541931.jpg'
x = txt.replace('../input/iais22-birds/submission_test/submission_test/', '')
x = x.split('.')
x[0]

In [35]:
#GUARDAR MODELO
model_path = 'transformer.pth'
torch.save(modelCNN2.state_dict(),model_path)

In [36]:
for 